# SeqAcademy Multiomics Tutorial

# Contents
1. Installation
    1. Set up channels
    2. Create an environment and install the packages
2. Alignment
    1. HISAT 
    2. Samtools
    3. MultiQC
3. ChIP-Seq
    1. MACS
    2. Bedtools
4. RNA-Seq
    1. HTSeq
    2. featureCounts
    3. DESeq
    4. Salmon
    5.dupRadar

# Installation

Before running any programs, we'll make sure that each software is installed correctly. This tutorial uses Bioconda (https://bioconda.github.io/). Bioconda is a channel for the conda package manager specializing in bioinformatics software. The available packages are listed here: https://bioconda.github.io/recipes.html#recipes.

## Set up channels

You will need to add the bioconda channel as well as the other channels bioconda depends on. It is important to add them in this order so that the priority is set correctly (that is, bioconda is highest priority).

The conda-forge channel contains many general-purpose packages not already found in the defaults channel. The r channel is only included due to backward compatibility. It is not mandatory, but without the r channel packages compiled against R 3.3.1 might not work.

This tutorial uses cells written in python and unix to perform its analyses. Lines that are written in unix are prefixed by an exclamation point. 

Select the following cell and run it. To run a cell, select the cell, click "Cell" the upper taskbar, and select "Run Cells". Or click the cell and press shift + enter.

Alternatively, the contents of any cell may be copy+pasted into the terminal emulator to run.

In [ ]:
!conda config --add channels defaults
!conda config --add channels conda-forge
!conda config --add channels bioconda

## Create an environment and install the packages

In this tutorial we will create an environment named "tutorial" and install the packages in there. Environments offer ways of installing packages in specific environments so they can be managed and run for different specifications. You can create, export, list, remove and update environments that have different versions of Python and/or packages installed in them. Switching or moving between environments is called activating the environment. You can also share an environment file.

This command will create an environment "tutorial" in which to install the packages used in this tutorial.

Run the following commands to create the environment. When it asks if you want to proceed, type "y" and press enter.

In [ ]:
!conda create -n tutorial hisat2 multiqc macs2 salmon bioconductor-deseq matplotlib ggplot pybedtools samtools bioconductor-rsamtools bedtools htseq subread bioconductor-rsubread deeptools

Then activate the environment with the following command.

In [ ]:
# For Mac and Linux

!source activate tutorial

# For Windows

!activate tutorial

# Alignment
## HISAT (Hierarchical Indexing for Spliced Alignment of Transcripts)

In this tutorial, we'll use Hisat to align the sample reads to a reference genome. Hisat automatically downloads and preprocesses the reads so they're ready to be aligned. Hisat (hierarchical indexing for spliced alignment of transcripts) is a highly efficient system for aligning reads from RNA sequencing experiments. HISAT uses an indexing scheme based on the Burrows-Wheeler transform and the Ferragina-Manzini (FM) index, employing two types of indexes for alignment: a whole-genome FM index to anchor each alignment and numerous local FM indexes for very rapid extensions of these alignments. HISAT’s hierarchical index for the human genome contains 48,000 local FM indexes, each representing a genomic region of ~64,000 bp.

The RNA-Seq data we'll use is from https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRP106028 and the ChIP-Seq data is from https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRP132584

The model organism for this project is Yeast i.e. Saccharomyces cerevisiae. For RNA-Seq, yeast data between euploid and aneuoploid conditions will be compared. For ChIP-SEq, yeast data between 3AT-treated and untreated conditions will be compared.

In [1]:
from pandas import read_csv

RNASeqSRARunTableFile='data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"]).astype(list)
RNASeqoutputSam = "test/" + RNASeqoutrun + ".sam"
RNASeqoutputAlignmentSummary = "test/" + RNASeqoutrun + ".txt"
RNASeqoutputMetrics = "test/" + RNASeqoutrun + ".metrics"
RNASeqoutputSortBam = "test/" + RNASeqoutrun + ".sorted.bam"

ChIPSeqSRARunTableFile='data/ChIPSeqSRA.tsv'
ChIPSeqSRATable = read_csv(ChIPSeqSRARunTableFile, delimiter='\t')
ChIPSeqoutrun = (ChIPSeqSRATable["Run"]).astype(list)
ChIPSeqoutputSam = "test/" + ChIPSeqoutrun + ".sam"
ChIPSeqoutputAlignmentSummary = "test/" + ChIPSeqoutrun + ".txt"
ChIPSeqoutputMetrics = "test/" + ChIPSeqoutrun + ".metrics"
ChIPSeqoutputSortBam = "test/" + ChIPSeqoutrun + ".sorted.bam"

In [ ]:
Then run the following command to create the yeast index.

In [ ]:
!bash scripts/make_yeast_index.sh

In [ ]:
Align the RNA-Seq samples using Hisat. 

In [ ]:
for index, individual in enumerate(RNASeqoutrun):
    run = RNASeqoutrun[index]
    summary = RNASeqoutputAlignmentSummary[index] 
    metrics = RNASeqoutputMetrics[index]
    sam = RNASeqoutputSam[index]
    bam = RNASeqoutputSortBam[index]
    !hisat2 -x yeast_index/genome --sra-acc $run --new-summary --summary-file $summary --met-file $metrics -S $sam

## Samtools 

We'll use samtools to sort the output files and convert them to bam files.

Sort the output files and convert them to bam files.

In [ ]:
for index, individual in enumerate(RNASeqoutrun):
    run = RNASeqoutrun[index]
    summary = RNASeqoutputAlignmentSummary[index] 
    metrics = RNASeqoutputMetrics[index]
    sam = RNASeqoutputSam[index]
    bam = RNASeqoutputSortBam[index]
    !samtools view -bSF4 $sam | samtools sort -o $bam
    

In [ ]:
Do the same thing for ChIP-Seq samples.

In [ ]:
for index, individual in enumerate(ChIPSeqoutrun):
    run = ChIPSeqoutrun[index]
    summary = ChIPSeqoutputAlignmentSummary[index] 
    metrics = ChIPSeqoutputMetrics[index]
    sam = ChIPSeqoutputSam[index]
    bam = ChIPSeqoutputSortBam[index]
    index = "yeast_index/genome"
    !hisat2 -x $index --sra-acc $run --new-summary --summary-file $summary --met-file $metrics -S $sam

In [ ]:
for index, individual in enumerate(ChIPSeqoutrun):
    run = ChIPSeqoutrun[index]
    summary = ChIPSeqoutputAlignmentSummary[index] 
    metrics = ChIPSeqoutputMetrics[index]
    sam = ChIPSeqoutputSam[index]
    bam = ChIPSeqoutputSortBam[index]
    !samtools view -bSF4 $sam | samtools sort -o $bam

## MultiQC

This section details quality control checks on the read data from either RNAseq or ChIPseq data using MultiQC. MultiQC takes all output and log files from an alignment software program and aggregates the information from all samples into one convenient report (html by default).

MultiQC was installed earlier in the tutorial, so all we need to do is run it on the data.

MultiQC is configured to run the same no matter what type of sequencing data is available, therefore the same command can be used to analyze either our RNAseq data or our ChIPseq data.  We include the option 'hisat_output' since we are aligning using the HISAT2 program.  See http://multiqc.info/docs/ for more information.

We use the 'hisat_output' option because we are analyzing data downloaded and aligned using the HISAT2 program.  We use the '--force' option to overwrite any previous versions of the multiqc_report.  '--quiet' only shows log warnings.

In [ ]:
!multiqc "".join(RNASeqoutrun) --quiet --outdir test/multiqc_rnaseq --force
!multiqc "".join(ChIPSeqoutrun) --quiet --outdir test/multiqc_chipseq --force

# ChIP-Seq
## MACS (Model-based Analysis for ChIP-Seq)

Peak-calling is one of the main steps scientists use in determining the locations where protein is bound in DNA. Peak detection software, such as MACS (Model-Based Analysis for ChIP-Seq), call peaks using the aligned sequecnes as input and returns precise locations of predicted peaks as output. In this tutorial, we'll use MACS.

More information about MACS: http://liulab.dfci.harvard.edu/MACS/Download.html

In [2]:
ChIPSeqControl = ChIPSeqSRATable.loc[ChIPSeqSRATable["source_name"] == "Untreated"]["Run"].astype(list)
ChIPSeqTreatment = ChIPSeqSRATable.loc[ChIPSeqSRATable["source_name"] != "Untreated"]["Run"].astype(list)

print("Control sample " + ChIPSeqControl)
print("Treatment sample " + ChIPSeqTreatment)

0    Control sample SRR6703656
2    Control sample SRR6703662
Name: Run, dtype: object
1    Treatment sample SRR6703661
3    Treatment sample SRR6703663
Name: Run, dtype: object


In [ ]:
for index, individual in enumerate(ChIPSeqControl):
    outputdirectory = "test/" + ChIPSeqTreatment.iloc[index]
    name = ChIPSeqTreatment.iloc[index]
    immunoprecipitate = "test/" + ChIPSeqTreatment.iloc[index] + ".sorted.bam"
    control = "test/" + ChIPSeqControl.iloc[index] + ".sorted.bam"
    !macs2 callpeak -c $control -t $immunoprecipitate -n $name --outdir $outputdirectory

INFO  @ Wed, 18 Apr 2018 16:11:33: 
# Command line: callpeak -c test/SRR6703656.sorted.bam -t test/SRR6703661.sorted.bam -n SRR6703661 --outdir test/SRR6703661
# ARGUMENTS LIST:
# name = SRR6703661
# format = AUTO
# ChIP-seq file = ['test/SRR6703661.sorted.bam']
# control file = ['test/SRR6703656.sorted.bam']
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 5.00e-02
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 1000 bps and 10000 bps
# Broad region calling is off
# Paired-End mode is off
 
INFO  @ Wed, 18 Apr 2018 16:11:33: #1 read tag files... 
INFO  @ Wed, 18 Apr 2018 16:11:33: #1 read treatment tags... 
INFO  @ Wed, 18 Apr 2018 16:11:33: Detected format is: BAM 
INFO  @ Wed, 18 Apr 2018 16:11:33: * Input file is gzipped. 
INFO  @ Wed, 18 Apr 2018 16:11:37:  1000000 
INFO  @ Wed, 18 Apr 2018 16:11:41:  2000000 
INFO  @ Wed, 18 Apr 2018 16:11:45:  3000000 
INFO  @ Wed, 18 Apr 2018 16:1

INFO  @ Wed, 18 Apr 2018 16:21:34:  8000000 
INFO  @ Wed, 18 Apr 2018 16:21:37:  9000000 
INFO  @ Wed, 18 Apr 2018 16:21:41:  10000000 
INFO  @ Wed, 18 Apr 2018 16:21:45:  11000000 
INFO  @ Wed, 18 Apr 2018 16:21:49:  12000000 
INFO  @ Wed, 18 Apr 2018 16:21:53:  13000000 
INFO  @ Wed, 18 Apr 2018 16:21:57:  14000000 
INFO  @ Wed, 18 Apr 2018 16:22:01:  15000000 
INFO  @ Wed, 18 Apr 2018 16:22:05:  16000000 
INFO  @ Wed, 18 Apr 2018 16:22:09:  17000000 
INFO  @ Wed, 18 Apr 2018 16:22:13:  18000000 
INFO  @ Wed, 18 Apr 2018 16:22:17:  19000000 
INFO  @ Wed, 18 Apr 2018 16:22:21:  20000000 
INFO  @ Wed, 18 Apr 2018 16:22:25:  21000000 
INFO  @ Wed, 18 Apr 2018 16:22:29:  22000000 
INFO  @ Wed, 18 Apr 2018 16:22:32:  23000000 
INFO  @ Wed, 18 Apr 2018 16:22:36:  24000000 
INFO  @ Wed, 18 Apr 2018 16:22:40:  25000000 
INFO  @ Wed, 18 Apr 2018 16:22:44:  26000000 
INFO  @ Wed, 18 Apr 2018 16:22:48:  27000000 
INFO  @ Wed, 18 Apr 2018 16:22:52:  28000000 
INFO  @ Wed, 18 Apr 2018 16:22:56:  

INFO  @ Wed, 18 Apr 2018 16:30:20: #3 Call peaks... 
INFO  @ Wed, 18 Apr 2018 16:30:20: #3 Pre-compute pvalue-qvalue table... 
INFO  @ Wed, 18 Apr 2018 16:30:58: #3 Call peaks for each chromosome... 
INFO  @ Wed, 18 Apr 2018 16:31:03: #4 Write output xls file... test/SRR6703661/SRR6703661_peaks.xls 
INFO  @ Wed, 18 Apr 2018 16:31:03: #4 Write peak in narrowPeak format file... test/SRR6703661/SRR6703661_peaks.narrowPeak 
INFO  @ Wed, 18 Apr 2018 16:31:03: #4 Write summits bed file... test/SRR6703661/SRR6703661_summits.bed 
INFO  @ Wed, 18 Apr 2018 16:31:03: Done! 
INFO  @ Wed, 18 Apr 2018 16:31:06: 
# Command line: callpeak -c test/SRR6703662.sorted.bam -t test/SRR6703663.sorted.bam -n SRR6703663 --outdir test/SRR6703663
# ARGUMENTS LIST:
# name = SRR6703663
# format = AUTO
# ChIP-seq file = ['test/SRR6703663.sorted.bam']
# control file = ['test/SRR6703662.sorted.bam']
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 5.00e-02
# Larger datase

INFO  @ Wed, 18 Apr 2018 16:39:44:  148000000 
INFO  @ Wed, 18 Apr 2018 16:39:48:  149000000 
INFO  @ Wed, 18 Apr 2018 16:39:51:  150000000 
INFO  @ Wed, 18 Apr 2018 16:39:55:  151000000 
INFO  @ Wed, 18 Apr 2018 16:39:58:  152000000 
INFO  @ Wed, 18 Apr 2018 16:40:02:  153000000 
INFO  @ Wed, 18 Apr 2018 16:40:05:  154000000 
INFO  @ Wed, 18 Apr 2018 16:40:09:  155000000 
INFO  @ Wed, 18 Apr 2018 16:40:12:  156000000 
INFO  @ Wed, 18 Apr 2018 16:40:16:  157000000 
INFO  @ Wed, 18 Apr 2018 16:40:19:  158000000 
INFO  @ Wed, 18 Apr 2018 16:40:23:  159000000 
INFO  @ Wed, 18 Apr 2018 16:40:26:  160000000 
INFO  @ Wed, 18 Apr 2018 16:40:30:  161000000 
INFO  @ Wed, 18 Apr 2018 16:40:33:  162000000 
INFO  @ Wed, 18 Apr 2018 16:40:37:  163000000 
INFO  @ Wed, 18 Apr 2018 16:40:40:  164000000 
INFO  @ Wed, 18 Apr 2018 16:40:44:  165000000 
INFO  @ Wed, 18 Apr 2018 16:40:47:  166000000 
INFO  @ Wed, 18 Apr 2018 16:40:50:  167000000 
INFO  @ Wed, 18 Apr 2018 16:40:54:  168000000 
INFO  @ Wed, 

For an in-depth discussion of what MACS2 does: https://github.com/taoliu/MACS/wiki/Advanced:-Call-peaks-using-MACS2-subcommands

## Bedtools

In this tutorial, we'll use Bedtools to extract the intersecting regions of the MACS output between the experimental conditions.

The Bedtools suite of programs is widely used for genomic interval manipulation or "genome algebra". pybedtools wraps and extends Bedtools and offers feature-level manipulations from within Python.

First we'll sort the output. The following line uses the `sort` command to sort the MACS output.

In [ ]:
!sort -k 1,1 -k2,2n test/SRR6703661/SRR6703661_peaks.narrowPeak > test/SRR6703661/SRR6703661_peaks.narrowPeak.sorted
!sort -k 1,1 -k2,2n test/SRR6703663/SRR6703663_peaks.narrowPeak > test/SRR6703661/SRR6703663_peaks.narrowPeak.sorted

Then we'll use pybedtools to truncate the output files to chromosome so no ends are out of bounds. 

In [ ]:
from pybedtools import BedTool

for index, individual in enumerate(ChIPSeqControl):
    outputdirectory = "test/" + ChIPSeqTreatment.iloc[index]
    name = ChIPSeqTreatment.iloc[index]
    sort = "test/" + name + "/" + name + "_peaks.narrowPeak.sorted"
    truncated = sort + ".truncated"    
    !BedTool($sort).truncate_to_chrom('sacCer3').saveas($truncated)

Then we'll find the intersecting regions between the experimental conditions and the control conditions. 

In [ ]:
!bedtools intersect -a test/SRR6703661/SRR6703661.narrowPeak.sorted.truncated -b test/SRR6703661/SRR6703663.narrowPeak.sorted.truncated -u > test/ChIPSeqintersect.bed

## Deeptools

deepTools is a suite of python tools particularly developed for the efficient analysis of high-throughput sequencing data, such as ChIP-seq, RNA-seq or MNase-seq. In this tutorial, we'll use it on ChIP-Seq data first. 

In [ ]:
for index, individual in enumerate(ChIPSeqControl):
    immunoprecipitatename = ChIPSeqTreatment.iloc[index]
    immunoprecipitate = "test/" + ChIPSeqTreatment.iloc[index] + ".sorted.bam"
    controlname = ChIPSeqControl.iloc[index]
    control = "test/" + ChIPSeqControl.iloc[index] + ".sorted.bam"
    output = "test/" + name + "/" + name + "_coverage.bw"
    !samtools index $immunoprecipitate
    !samtools index $control
    !bamCoverage --normalizeUsingRPKM -b $immunoprecipitate -o $output
    !bamCoverage --normalizeUsingRPKM -b $control -o 

# RNA-Seq
## HTSeq (High-through sequencing)

HTSeq is a Python library to facilitate the rapid development of RNA-Seq analysis. HTSeq offers parsers for many common data formats in HTS projects, as well as classes to represent data, such as genomic coordinates, sequences, sequencing reads, alignments, gene model information and variant calls, and provides data structures that allow for querying via genomic coordinates. In this tutorial we will use htseq-count, a tool developed with HTSeq that preprocesses RNA-Seq data for differential expression analysis by counting the overlap of reads with genes.

In [ ]:
from pandas import read_csv

gtf = "data/Saccharomyces_cerevisiae.R64-1-1.84.gtf"

RNASeqSRARunTableFile='data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"]).astype(list)
RNASeqoutputSortBam = "test/" + RNASeqoutrun + ".sorted.bam"

In [ ]:
for index, individual in enumerate(RNASeqoutputSortBam):
    input = individual
    output = individual + ".genecount.txt"
    !htseq-count -m intersection-nonempty -s no -f bam $input $gtf > $output

## featureCounts

featureCounts is a highly efficient general-purpose read summarization program that counts mapped reads for genomic features such as genes, exons, promoter, gene bodies, genomic bins and chromosomal locations. It can be used to count both RNA-seq and genomic DNA-seq reads. It is available in the SourceForge Subread package or the Bioconductor Rsubread package.

In [ ]:
for index, individual in enumerate(RNASeqoutputSortBam):
    input = individual
    !featureCounts -s -a $gtf -o test/featureCounts $input

## DESeq (Differential Expression Sequencing)

Estimate variance-mean dependence in count data from high-throughput sequencing assays and test for differential expression based on a model using the negative binomial distribution

In [ ]:
!Rscript scripts/runDeseq.R

## Salmon

Salmon is a tool for quantifying the expression of transcripts using RNA-seq data. Salmon uses new algorithms (specifically, coupling the concept of quasi-mapping with a two-phase inference procedure) to provide accurate expression estimates very quickly (i.e. wicked-fast) and while using little memory. Salmon performs its inference using an expressive and realistic model of RNA-seq data that takes into account experimental attributes and biases commonly observed in real RNA-seq data.

In [ ]:
!wget -O data/transcript.fa.gz ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_rna.fna.gz

In [ ]:
Then unzip the transcript.fa.gz file.

In [ ]:
!gunzip data/transcript.fa.gz

In [ ]:
Finally, create the salmon index and run salmon on it.

In [ ]:
salmon_index = "test/yeast-salmon.idx"

!salmon index -t data/transcript.fa -i $salmon_index --type quasi -k 31


In [ ]:
from pandas import read_csv

RNASeqSRARunTableFile='data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"])
RNASeqBam = "test/" + RNASeqoutrun + ".sorted.bam"

for index, individual in enumerate(RNASeqBam):
    !salmon quant --targets $salmon_index -l A --output test -a $individual

## dupRadar
dupRadar provides an easy way to distinguish between artifactual vs natural duplicate reads in RNA-Seq data. Prior to dupRadar only global duplication rates were used and they don't take into account the effect of gene expression levels. dupRadar relates duplication rates and length normalized read counts of every gene to model the dependency of both variables.

In [ ]:
for index, individual in enumerate(RNASeqoutputSortBam):
    !Rscript scripts/dupRadar.R $individual $gtf